# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [14]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json
import gmaps
import os

!jupyter nbextension enable --py --sys-prefix widgetsnbextension
!pip install gmaps
!jupyter nbextension enable --py --sys-prefix gmaps
!jupyter nbextension enable --py widgetsnbextension --sys-prefix

# Import API key
from api_keys import g_key


Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


Enabling notebook extension jupyter-gmaps/extension...
      - Validating: OK
Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [15]:
weather_df = pd.read_csv('output_data/cities.csv')

weather_df.head()

,City,Country,Date,Cloudiness,Humidity,Latitude,Longitude,Max Temp,Wind Speed
0,Noblesville,US,1592584415,0,45,40.05,-86.01,88.0,1.01
1,Cabo San Lucas,MX,1592583924,5,45,22.89,-109.91,93.2,3.36
2,George Town,MY,1592583915,20,94,5.41,100.34,80.6,2.08
3,Punta Arenas,CL,1592584510,90,100,-53.15,-70.92,33.8,12.75
4,Ponta do Sol,PT,1592583915,40,73,32.67,-17.10,71.6,13.87


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [16]:
gmaps.configure(api_key=g_key)

In [17]:
# Store latitude and longitude in locations
locations = weather_df[["Latitude", "Longitude"]]

# Fill NaN values and convert to float
humidity = weather_df["Humidity"].astype(float)


In [18]:
# Plot Heatmap
fig = gmaps.figure(center=(14.5994, 28.6731), zoom_level=2)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=80,
                                 point_radius=3)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [19]:
#A max temperature lower than 80 degrees but higher than 70.
#Wind speed less than 10 mph.
#Zero cloudiness.
#Drop any rows that don't contain all three conditions. 

In [20]:
narrowed_city_df = weather_df.loc[(weather_df["Max Temp"] > 70) & (weather_df["Max Temp"] < 80) & (weather_df["Wind Speed"] < 10) & (weather_df["Cloudiness"] == 0)]
narrowed_city_df.head(10)


,City,Country,Date,Cloudiness,Humidity,Latitude,Longitude,Max Temp,Wind Speed
70,Karratha,AU,1592584243,0,45,-20.74,116.85,71.78,6.26
73,Vila,PT,1592583966,0,63,42.03,-8.16,75.00,9.17
74,Arganil,PT,1592584522,0,55,40.22,-8.05,72.00,5.01
85,Srednekolymsk,RU,1592584337,0,36,67.45,153.68,72.68,6.96
160,Roebourne,AU,1592584534,0,47,-20.78,117.13,71.47,5.06
163,Henties Bay,NaN,1592584535,0,36,-22.12,14.28,78.01,9.98
206,Lata,PT,1592583975,0,55,40.16,-8.33,72.00,5.01
300,Saldanha,ZA,1592584557,0,35,-33.01,17.94,71.60,9.17
431,San Policarpo,PH,1592584656,0,82,12.18,125.51,79.72,4.29
592,Manokwari,ID,1592584691,0,70,-0.87,134.08,79.77,3.58


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [21]:
hotel_df = narrowed_city_df.copy()
hotel_df["Hotel Name"] = ""
hotel_df.head()

,City,Country,Date,Cloudiness,Humidity,Latitude,Longitude,Max Temp,Wind Speed,Hotel Name
70,Karratha,AU,1592584243,0,45,-20.74,116.85,71.78,6.26,
73,Vila,PT,1592583966,0,63,42.03,-8.16,75.00,9.17,
74,Arganil,PT,1592584522,0,55,40.22,-8.05,72.00,5.01,
85,Srednekolymsk,RU,1592584337,0,36,67.45,153.68,72.68,6.96,
160,Roebourne,AU,1592584534,0,47,-20.78,117.13,71.47,5.06,


In [22]:
# params dictionary to update each iteration
params = {
    "radius": 50000,
    "types": "hotel",
    "keyword": "hotel",
    "key": g_key
}

# Use the lat/lng we recovered to identify hotels
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Latitude"]
    lng = row["Longitude"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "Hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)

    # convert to json
    name_address = name_address.json()
    # print(json.dumps(name_address, indent=4, sort_keys=True))

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

Missing field/result... skipping.


In [23]:
hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
hotel_df

,City,Country,Date,Cloudiness,Humidity,Latitude,Longitude,Max Temp,Wind Speed,Hotel Name
70,Karratha,AU,1592584243,0,45,-20.74,116.85,71.78,6.26,ibis Styles Karratha
73,Vila,PT,1592583966,0,63,42.03,-8.16,75.00,9.17,Ribeira Collection Hotel by Piamonte Hotels
74,Arganil,PT,1592584522,0,55,40.22,-8.05,72.00,5.01,Hotel Solneve
85,Srednekolymsk,RU,1592584337,0,36,67.45,153.68,72.68,6.96,NaN
160,Roebourne,AU,1592584534,0,47,-20.78,117.13,71.47,5.06,ibis Styles Karratha
163,Henties Bay,NaN,1592584535,0,36,-22.12,14.28,78.01,9.98,Cape Cross Lodge
206,Lata,PT,1592583975,0,55,40.16,-8.33,72.00,5.01,Bussaco Palace Hotel
300,Saldanha,ZA,1592584557,0,35,-33.01,17.94,71.60,9.17,Paternoster Accommodation
431,San Policarpo,PH,1592584656,0,82,12.18,125.51,79.72,4.29,Hotel Europa
592,Manokwari,ID,1592584691,0,70,-0.87,134.08,79.77,3.58,ASTON Niu Manokwari Hotel and Conference Center


In [24]:
final_hotel_df = hotel_df.dropna(how='any')
final_hotel_df

,City,Country,Date,Cloudiness,Humidity,Latitude,Longitude,Max Temp,Wind Speed,Hotel Name
70,Karratha,AU,1592584243,0,45,-20.74,116.85,71.78,6.26,ibis Styles Karratha
73,Vila,PT,1592583966,0,63,42.03,-8.16,75.00,9.17,Ribeira Collection Hotel by Piamonte Hotels
74,Arganil,PT,1592584522,0,55,40.22,-8.05,72.00,5.01,Hotel Solneve
160,Roebourne,AU,1592584534,0,47,-20.78,117.13,71.47,5.06,ibis Styles Karratha
206,Lata,PT,1592583975,0,55,40.16,-8.33,72.00,5.01,Bussaco Palace Hotel
300,Saldanha,ZA,1592584557,0,35,-33.01,17.94,71.60,9.17,Paternoster Accommodation
431,San Policarpo,PH,1592584656,0,82,12.18,125.51,79.72,4.29,Hotel Europa
592,Manokwari,ID,1592584691,0,70,-0.87,134.08,79.77,3.58,ASTON Niu Manokwari Hotel and Conference Center


In [25]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in final_hotel_df.iterrows()]
locations = final_hotel_df[["Latitude", "Longitude"]]

In [26]:
# Add marker layer ontop of heat map
# markers = gmaps.marker_layer(locations)

markers = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(markers)

fig

# Display Map

Figure(layout=FigureLayout(height='420px'))